In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from environment import *

import analysis as a

p = make_paths(a)

In [ ]:
signal_matrix = pd.read_table(p['signal_matrix_file_path'], index_col=0)

signal_matrix.fillna(0, inplace=True)

signal_matrix = ccal.drop_df_slice_greedily(
    signal_matrix, min_n_not_na_unique_value=2)

In [ ]:
w_file_path = p['w_file_path']

h_file_path = p['h_file_path']

if not (os.path.isfile(w_file_path) and os.path.isfile(h_file_path)):

    k_nmf, k_nmfcc_distance_linkage_ccc, nmfcc__k_x_column = ccal.nmf_consensus_cluster_with_multiple_k(
        signal_matrix,
        a.NMF_KS,
        n_job=1,
        n_clustering=10,
        plot_df=signal_matrix.size < 1e6,
        plot_w=signal_matrix.shape[0] < 1600,
        plot_h=signal_matrix.shape[1] < 1600,
        directory_path=p['nmf_directory_path'])

In [ ]:
w = pd.read_table(w_file_path, index_col=0)

h = pd.read_table(h_file_path, index_col=0)

for wt_or_h, wt_or_h_matrix in (('wt', w.T), ('h', h)):

    title = 'NMF k={} {}'.format(a.NMF_K, wt_or_h)

    ccal.plot_heat_map(
        wt_or_h_matrix,
        normalization_method='-0-',
        normalization_axis=0,
        cluster_axis=1,
        title=title,
        xaxis_title=wt_or_h_matrix.columns.name,
        yaxis_title=wt_or_h_matrix.index.name)

In [ ]:
for wt_or_h, targets, features in (
    ('wt', w.T, signal_matrix.T),
    ('h', h, signal_matrix),
):

    targets = targets**0.32

    ccal.make_match_panels(
        ((targets).iloc[i] for i in range(targets.shape[0])), {
            'Signal': {
                'df': features,
                'emphasis': 'high',
                'data_type': 'continuous'
            }
        },
        n_job=1,
        extreme_feature_threshold=16,
        directory_path=p['{}_match_directory_path'.format(wt_or_h)],
        plotly_directory_path=p['{}_match_plotly_directory_path'.format(
            wt_or_h)])